In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
from tqdm import tqdm_notebook as tqdm # progress bar
import matplotlib.pyplot as plt

import os, json, cv2, random
import skimage.io as io
import copy
import pickle
from pathlib import Path
from typing import Optional
from tqdm import tqdm

# torch
import torch



# Albumenatations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

#from pycocotools.coco import COCO
from sklearn.model_selection import StratifiedKFold

# glob
from glob import glob

# numba
import numba
from numba import jit

import warnings
warnings.filterwarnings('ignore') #Ignore "future" warnings and Data-Frame-Slicing warnings.


# detectron2
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

setup_logger()

In [ ]:
image=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')

In [ ]:
!mkdir image_dir
!conda install gdcm -c conda-forge -y
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut


In [ ]:
!mkdir train
!tar -C train -zxvf ../input/jpg-data/train.tar.gz

In [ ]:
train=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')

In [ ]:
meta=pd.read_csv('../input/jpg-data/meta.csv')

In [ ]:
def get_size(image_id):
    row =meta[meta['image_id']==image_id]
    return row['dim0'].iloc[0],row['dim1'].iloc[0]

In [ ]:
def get_dict(xyz):
    dataset_dicts=[]
    for i in range(len(train)):
        row = train.iloc[i]
        record = {}
        record['file_name']='train/'+row['id'].split('_')[0]+'.jpg'
        record['image_id']=row['id'].split('_')[0]
        x,y=get_size(row['id'].split('_')[0])
        record['width']=670
        record['height']=670
        resized_height=670
        resized_width=670
        if row['label'].split()[0]=='opacity':
            objs=[]
            if len(row['label'].split())>1:
                for j in row['label'].split('opacity'):
                    h_ratio = resized_height / y
                    w_ratio = resized_width / x
                    l=j.split()
                    if len(l)>1:
                        l=l[1:]
                        l=[float(k) for k in l ]
                        l[0]=l[0]*w_ratio
                        l[2]=l[2]*w_ratio
                        l[1]=l[1]*h_ratio
                        l[3]=l[3]*h_ratio
                        obj = {
                        "bbox": l,
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "category_id": 0,
                        }
                        objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ['train']:
    DatasetCatalog.register(""+d , lambda d=d: get_dict("" + d))
    MetadataCatalog.get('train').set(thing_classes=["o"])


In [ ]:
balloon_metadata = MetadataCatalog.get("train")
dataset_dicts_train = DatasetCatalog.get("train")

In [ ]:
cfg = get_cfg()
config_name = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml" 
#config_name = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
#config_name = "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"

cfg.merge_from_file(model_zoo.get_config_file(config_name))

cfg.DATASETS.TRAIN = ("train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_name)
cfg.MODEL.WEIGHTS="../input/1siim-covid19-detectron2-weights/output/model_final.pth"


cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025

cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1000 #adjust up if val mAP is still rising, adjust down if overfit
#cfg.SOLVER.STEPS = (100, 500) # must be less than  MAX_ITER 
#cfg.SOLVER.GAMMA = 0.05


cfg.SOLVER.CHECKPOINT_PERIOD = 100000  # Small value=Frequent save need a lot of storage.
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


#Training using custom trainer defined above
#trainer = AugTrainer(cfg) 
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
dir(trainer)

In [ ]:
torch.save(trainer.state_dict(),'check.pth')